In [10]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import plotly.io as pio
import plotly.colors as colors
pio.templates.default = "plotly_white"

Now that we have the proper libraries downlaoded, we must read the data and view the head to ensure the file downloaded correctly and for initial data analysis

In [11]:
store_data = pd.read_csv('Sample - Superstore.csv', encoding='latin-1')
print(store_data.head())

   Row ID        Order ID  Order Date   Ship Date       Ship Mode Customer ID  \
0       1  CA-2016-152156   11/8/2016  11/11/2016    Second Class    CG-12520   
1       2  CA-2016-152156   11/8/2016  11/11/2016    Second Class    CG-12520   
2       3  CA-2016-138688   6/12/2016   6/16/2016    Second Class    DV-13045   
3       4  US-2015-108966  10/11/2015  10/18/2015  Standard Class    SO-20335   
4       5  US-2015-108966  10/11/2015  10/18/2015  Standard Class    SO-20335   

     Customer Name    Segment        Country             City  ...  \
0      Claire Gute   Consumer  United States        Henderson  ...   
1      Claire Gute   Consumer  United States        Henderson  ...   
2  Darrin Van Huff  Corporate  United States      Los Angeles  ...   
3   Sean O'Donnell   Consumer  United States  Fort Lauderdale  ...   
4   Sean O'Donnell   Consumer  United States  Fort Lauderdale  ...   

  Postal Code  Region       Product ID         Category Sub-Category  \
0       42420   Sout

View dimensions and column names of our data sheet.

In [12]:
print(store_data.shape)
store_data.columns.values

(9994, 21)


array(['Row ID', 'Order ID', 'Order Date', 'Ship Date', 'Ship Mode',
       'Customer ID', 'Customer Name', 'Segment', 'Country', 'City',
       'State', 'Postal Code', 'Region', 'Product ID', 'Category',
       'Sub-Category', 'Product Name', 'Sales', 'Quantity', 'Discount',
       'Profit'], dtype=object)

As of now, we have 21 total categories and 9994 total purchases that we can analyze.

*** NOTE *** There are 9994 total purchases in this data set but each purchase can have multiple sales of the same item.

We must clean the data and then begin our true analysis.

In [13]:
store_data.isnull().values.any()

False

Our data came without any null values, in which we can now bein our analysis.

First, we must take a look at dates over the course of months, days of the month, and days of the week.

In [22]:
store_data['Order Date'] = pd.to_datetime(store_data['Order Date'])
store_data['Ship Date'] = pd.to_datetime(store_data['Ship Date']) 

store_data['Order Month'] = store_data['Order Date'].dt.month 
store_data['Order Year'] = store_data['Order Date'].dt.year
store_data['Order Day of Week'] = store_data['Order Date'].dt.dayofweek

# Create sales by month category and display it over the course of the year
sales_by_month = store_data.groupby('Order Month')['Sales'].sum().reset_index()
sales_by_month_figure = px.line(sales_by_month, x = 'Order Month', y = 'Sales',
                                title = 'Number of sales per month')
sales_by_month_figure.show()

sales_by_year = store_data.groupby('Order Year')['Sales'].sum().reset_index()
sales_by_year_figure = px.line(sales_by_year, x = 'Order Year', y = 'Sales',
                                title = 'Number of sales per year')
sales_by_year_figure.show()

sales_by_day = store_data.groupby('Order Day of Week')['Sales'].sum().reset_index()
sales_by_day_figure = px.line(sales_by_day, x = 'Order Day of Week', y = 'Sales',
                                title = 'Number of sales per day')
sales_by_day_figure.show()

Now that we have information regarding total sales over time, we must look within each category to determine both revenue & profit.

In [24]:
sales_by_category = store_data.groupby('Category')['Sales'].sum().reset_index()

sales_by_category_figure = px.pie(sales_by_category,
                                  values='Sales',
                                  names='Category',
                                  hole=.5,
                                  color_discrete_sequence=px.colors.qualitative.Pastel)
sales_by_category_figure.update_traces(textposition='inside', textinfo='percent+label')
sales_by_category_figure.update_layout(title_text='Sales Analysis by Category', title_font=dict(size=24))

sales_by_category_figure.show()

sales_by_subcategory = store_data.groupby('Sub-Category')['Sales'].sum().reset_index()
sales_by_subcategory_figure = px.bar(sales_by_subcategory, 
             x='Sub-Category', 
             y='Sales', 
             title='Sales Analysis by Sub-Category')
sales_by_subcategory_figure.show()

Now that we can see the revenue, we must look at the profit of each category and subcategory.

In [26]:
profit_by_category = store_data.groupby('Category')['Profit'].sum().reset_index()

profit_by_category_figure = px.pie(profit_by_category, 
             values='Profit', 
             names='Category', 
             hole=0.5, 
             color_discrete_sequence=px.colors.qualitative.Pastel)

profit_by_category_figure.update_traces(textposition='inside', textinfo='percent+label')
profit_by_category_figure.update_layout(title_text='Profit Analysis by Category', title_font=dict(size=24))

profit_by_category_figure.show()

profit_by_subcategory = store_data.groupby('Sub-Category')['Profit'].sum().reset_index()
profit_by_subcategory_figure = px.bar(profit_by_subcategory, x='Sub-Category', 
             y='Profit', 
             title='Profit Analysis by Sub-Category')
profit_by_subcategory_figure.show()

In [31]:
sales_profit_category = store_data.groupby('Segment').agg({'Sales': 'sum', 'Profit': 'sum'}).reset_index()
color_palette = colors.qualitative.Pastel

sales_profit_category_figure = go.Figure()
sales_profit_category_figure.add_trace(go.Bar(x=sales_profit_category['Segment'], 
                     y=sales_profit_category['Sales'], 
                     name='Sales',
                     marker_color=color_palette[0]))
sales_profit_category_figure.add_trace(go.Bar(x=sales_profit_category['Segment'], 
                     y=sales_profit_category['Profit'], 
                     name='Profit',
                     marker_color=color_palette[1]))

sales_profit_category_figure.update_layout(title='Sales and Profit Analysis by Customer Segment',
                  xaxis_title='Customer Segment', yaxis_title='Amount')

sales_profit_category_figure.show()

# Create comparison for subcategories
sales_profit_subcategory = store_data.groupby('Sub-Category').agg({'Sales': 'sum', 'Profit': 'sum'}).reset_index()

sales_profit_subcategory_fig = go.Figure()
sales_profit_subcategory_fig.add_trace(go.Bar(x=sales_profit_subcategory['Sub-Category'], 
                     y=sales_profit_subcategory['Sales'], 
                     name='Sales',
                     marker_color=color_palette[0]))
sales_profit_subcategory_fig.add_trace(go.Bar(x=sales_profit_subcategory['Sub-Category'], 
                     y=sales_profit_subcategory['Profit'], 
                     name='Profit',
                     marker_color=color_palette[1]))

sales_profit_subcategory_fig.update_layout(title='Sales and Profit Analysis by Customer Segment',
                  xaxis_title='Customer Segment', yaxis_title='Amount')

sales_profit_subcategory_fig.show()

In [36]:
# Find each segement sales to profit ratio
sales_profit_category_ratio = store_data.groupby('Segment').agg({'Sales': 'sum', 'Profit': 'sum'}).reset_index()
sales_profit_category_ratio['Sales_to_Profit_Ratio'] = sales_profit_category_ratio['Sales'] / sales_profit_category_ratio['Profit']
print(sales_profit_category_ratio[['Segment', 'Sales_to_Profit_Ratio']])
print('\n\n\n')

# Find each subcategory sales to profit ratio
sales_profit_subcategory_ratio = store_data.groupby('Sub-Category').agg({'Sales': 'sum', 'Profit': 'sum'}).reset_index()
sales_profit_subcategory_ratio['Sales_to_Profit_Ratio'] = sales_profit_subcategory_ratio['Sales'] / sales_profit_subcategory_ratio['Profit']
print(sales_profit_subcategory_ratio[['Sub-Category', 'Sales_to_Profit_Ratio']])



       Segment  Sales_to_Profit_Ratio
0     Consumer               8.659471
1    Corporate               7.677245
2  Home Office               7.125416




   Sub-Category  Sales_to_Profit_Ratio
0   Accessories               3.991267
1    Appliances               5.928555
2           Art               4.154362
3       Binders               6.730671
4     Bookcases             -33.082259
5        Chairs              12.352277
6       Copiers               2.688491
7     Envelopes               2.365879
8     Fasteners               3.185068
9   Furnishings               7.022295
10       Labels               2.251305
11     Machines              55.909076
12        Paper               2.304581
13       Phones               7.413268
14      Storage              10.519547
15     Supplies             -39.251163
16       Tables             -11.676159


Now that the ratio of categories/sub-categories and their respective Sales to Profit ratio has been established, we should analyze the data over time to determine potential trends or patterns that could make the company more money.

Find the top 20 most profitable products and their respective ratio

In [62]:
# sales_profit_product_ratio = store_data.groupby('Product Name').agg({'Sales': 'sum', 'Profit': 'sum'}).reset_index()
# store_data['Sales_to_Profit_Ratio'] = sales_profit_product_ratio['Sales'] / sales_profit_product_ratio['Profit']

# Profit_info = store_data[["Product Name", "Profit", "Sales", "Sales_to_Profit_Ratio"]].sort_values(by=['Profit'], ascending = False)
# Profit_info.head(20)

products_info = store_data.groupby(['Product Name']).sum().reset_index()
Profit_info = store_data[["Product Name", "Profit"]].sort_values(by=['Profit'], ascending = False)
Profit_info.head(10)


TypeError: datetime64 type does not support sum operations